In [ ]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

!pip install rdkit-pypi

In [ ]:
!pip install rdkit
!pip install selfies
!pip install transformers datasets

In [ ]:
# Download SA Score script (by Ertl)
!wget https://raw.githubusercontent.com/rdkit/rdkit/master/Contrib/SA_Score/sascorer.py
!wget https://github.com/rdkit/rdkit/raw/master/Contrib/SA_Score/fpscores.pkl.gz

In [ ]:
import random, os, numpy as np, pandas as pd

def seed_everything(seed=42):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

In [ ]:
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

In [ ]:
df = pd.read_csv('USPTO_50K.csv')
df

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
import sascorer
import numpy as np
from tqdm.notebook import tqdm  # or tqdm.auto for non-Colab use

# Enable tqdm with pandas apply
tqdm.pandas()

# If not already loaded:
# df = pd.read_csv("your_uspto_50k.csv")  # make sure it has a 'reactions' column

def compute_difficulty_metrics(reaction_smiles):
    try:
        product_smiles = reaction_smiles.split(">")[-1]
        mol = Chem.MolFromSmiles(product_smiles)
        if mol is None:
            return np.nan, np.nan, np.nan, np.nan

        sa_score = sascorer.calculateScore(mol)
        ring_count = mol.GetRingInfo().NumRings()
        heavy_atom_count = Descriptors.HeavyAtomCount(mol)

        difficulty_score = (
            0.5 * sa_score +
            0.3 * ring_count +
            0.2 * heavy_atom_count
        )

        return sa_score, ring_count, heavy_atom_count, difficulty_score

    except Exception:
        return np.nan, np.nan, np.nan, np.nan

# Apply with progress bar
df[['sa_score', 'ring_count', 'heavy_atoms', 'difficulty']] = df['reactions'].progress_apply(
    compute_difficulty_metrics
).apply(pd.Series)

# Remove problematic rows and sort
df = df.dropna(subset=['difficulty']).sort_values('difficulty').reset_index(drop=True)

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(8, 5))
# sns.histplot(df_sorted['difficulty'], bins=40, kde=True)
# plt.title("Distribution of Retrosynthesis Difficulty Scores")
# plt.xlabel("Difficulty Score")
# plt.ylabel("Frequency")
# plt.grid(True)
# plt.show()

In [ ]:
from rdkit import Chem

def canonicalize_smiles(smi):
    """Canonicalizes multi-fragment SMILES (e.g. 'CC.O') with dot handling."""
    parts = smi.split('.')
    mols = [Chem.MolFromSmiles(part) for part in parts]
    if any(m is None for m in mols):
        return smi  # fallback if any fragment fails
    canonical_parts = [Chem.MolToSmiles(m, canonical=True) for m in mols]
    return '.'.join(sorted(canonical_parts))  # sort ensures consistent order

In [ ]:
# Split product/reactants from reaction SMILES
df['product'] = df['reactions'].apply(lambda x: x.split('>')[-1])
df['reactants'] = df['reactions'].apply(lambda x: x.split('>')[0])

# Canonicalize both sides
df['product'] = df['product'].apply(canonicalize_smiles)
df['reactants'] = df['reactants'].apply(canonicalize_smiles)

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)  # 81/9/10

train_df = train_df.sort_values(by="difficulty").reset_index(drop=True)

In [ ]:
# q1 = train_df['difficulty'].quantile(1/3)
# q2 = train_df['difficulty'].quantile(2/3)

# print(f"Easy ≤ {q1:.2f}, Medium ≤ {q2:.2f}, Hard > {q2:.2f}")

# easy_df = train_df[train_df["difficulty"] <= q1]
# medium_df = train_df[(train_df["difficulty"] > q1) & (train_df["difficulty"] <= q2)]
# hard_df = train_df[train_df["difficulty"] > q2]

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments, AutoTokenizer
from datasets import Dataset

t5_tokenizer = AutoTokenizer.from_pretrained("sagawa/ReactionT5v2-retrosynthesis")

# Add pad token if missing
if t5_tokenizer.pad_token is None:
    t5_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def tokenize_function(example):
    # Encoder input = product SMILES, tokenized with ChemBERTa
    input_ = t5_tokenizer(
        "retro: " + example['product'],
        padding="max_length",
        truncation=True,
        max_length=128
    )

    # Decoder target = reactants SMILES, tokenized with GPT2 tokenizer
    target = t5_tokenizer(
        example['reactants'],
        padding="max_length",
        truncation=True,
        max_length=128
    )

    input_['labels'] = target['input_ids']
    return input_

train_ds = Dataset.from_pandas(train_df[['product', 'reactants']])
val_ds = Dataset.from_pandas(val_df[['product', 'reactants']])
test_ds = Dataset.from_pandas(test_df[['product', 'reactants']])

train_tokenized = train_ds.map(tokenize_function, batched=False)
val_tokenized = val_ds.map(tokenize_function, batched=False)
test_tokenized = test_ds.map(tokenize_function, batched=False)

In [ ]:
def get_curriculum_indices(epoch, total_epochs, dataset_length, min_frac=0.1, max_frac=1.0):
    '''
    Pacing function as per Hacohen and Weinshall
    '''
    frac = min(1.0, (epoch + 1) / (0.5 * total_epochs))  # Full dataset by halfway point
    return list(range(int(dataset_length * frac)))

In [ ]:
from transformers import EncoderDecoderModel, Trainer, TrainingArguments, AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import os

# Set output path
output_dir = "CurriculumLearning/retrosyn-t5-curriculum"

model = AutoModelForSeq2SeqLM.from_pretrained("sagawa/ReactionT5v2-retrosynthesis")

model.config.pad_token_id = t5_tokenizer.pad_token_id

checkpoints = sorted(
    [ckpt for ckpt in os.listdir(output_dir) if ckpt.startswith("checkpoint-epoch")],
    key=lambda x: int(x.split("-")[-1])
)

resume_checkpoint = os.path.join(output_dir, checkpoints[-1]) if checkpoints else None

if resume_checkpoint:
    print("Resuming from checkpoint:", resume_checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(resume_checkpoint)
else:
    print("No checkpoints found. Starting from scratch.")

# TrainingArguments
training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="epoch",
    save_strategy="no",  # no automatic save
    per_device_train_batch_size=64,
    per_device_eval_batch_size=8,
    eval_accumulation_steps=2,
    prediction_loss_only=False,
    num_train_epochs=1,  # we manually loop
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    load_best_model_at_end=False,
    fp16=True
)

In [ ]:
from transformers import Trainer
from tqdm import tqdm

class CurriculumTrainer(Trainer):
    def set_dataset(self, dataset):
        self.train_dataset = dataset

# ✅ Initialize trainer once with dummy dataset
trainer = CurriculumTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized.select([0]),
    tokenizer=t5_tokenizer
)

total_epochs = 100
epoch_bar = tqdm(range(total_epochs), desc="Curriculum Training", unit="epoch")

for epoch in epoch_bar:
    indices = get_curriculum_indices(epoch, total_epochs, len(train_tokenized))
    current_data = train_tokenized.select(indices)

    # ✅ Set new dataset
    trainer.set_dataset(current_data)

    # ✅ Train for *one curriculum step* (not full 100 epochs again!)
    trainer.args.num_train_epochs = 1
    trainer.state.epoch = 0  # 🔁 Reset internal epoch count for logging

    trainer.train(resume_from_checkpoint=None)

    # ✅ Nice progress bar update
    epoch_bar.set_description(f"Epoch {epoch+1:03d} | Samples: {len(indices)}")

    # 💾 Save checkpoints
    if (epoch + 1) % 20 == 0:
        checkpoint_path = os.path.join(output_dir, f"checkpoint-epoch-{epoch+1}")
        model.save_pretrained(checkpoint_path)
        t5_tokenizer.save_pretrained(checkpoint_path)
        epoch_bar.write(f"💾 Saved checkpoint at {checkpoint_path}")